In [1]:
!pip install fair-esm


  Using cached fair_esm-2.0.0-py3-none-any.whl.metadata (37 kB)
Using cached fair_esm-2.0.0-py3-none-any.whl (93 kB)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
import esm

# Load the pretrained model
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
model.eval()  # Set to inference mode

batch_converter = alphabet.get_batch_converter()

# Example protein sequence (FASTA format: list of tuples)
data = [("protein1", "MKTLLILAVAFASAGGQWQLT")]

# Convert to batch tensors
batch_labels, batch_strs, batch_tokens = batch_converter(data)

# Forward pass to get logits (amino acid scores per position)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=False)

# Extract log probabilities
log_probs = torch.nn.functional.log_softmax(results["logits"], dim=-1)

# Example: Mutation at position 10 (0-based: token 11 due to BOS token)
pos = 10
wt_aa = batch_tokens[0, pos + 1]
mut_aa = alphabet.get_idx("A")  # Substitute with alanine

# Log-likelihood ratio (LLR) score
llr = log_probs[0, pos + 1, wt_aa] - log_probs[0, pos + 1, mut_aa]
print(f"LLR for mutation at position {pos+1}: {llr.item():.4f}")


Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1b_t33_650M_UR50S.pt" to /Users/doma/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm1b_t33_650M_UR50S-contact-regression.pt" to /Users/doma/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S-contact-regression.pt
LLR for mutation at position 11: 2.0006
